<a href="https://colab.research.google.com/github/zahranorozzadeh/DeepLearning45/blob/main/gender_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle


In [ ]:
!pip install retina_face
!pip3 install  wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-3.14.0-py3-none-any.whl size=10055 sha256=1177114ed41ad03dae70018df1930a1ec95fee52b9e512c7c1ac59961b31a856
  Stored in directory: /root/.cache/pip/wheels/ee/cf/ac/e3ff09ceeea4bf7ff64b04de426caf3a8d0d1a299346ca54fd
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 3.6.4
    Uninstalling gdown-3.6.4:
      Successfully uninstalled gdown-3.6.4
     |████████████████████████████████| 1.7 MB 2.3 MB/s 
     |████████████████████████████████| 97 kB 6.1 MB/s 
     |████████████████████████████████| 180 kB 41.8 MB/s 
     |████████████████████████████████| 139 kB 41.2 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=eaea1123e95f52964b6a63301da30b6c2a5a73e0dea0aadbf46dd5022d0c5260

In [ ]:
!kaggle datasets download -d ashishjangra27/gender-recognition-200k-images-celeba

100% 1.31G/1.32G [00:12<00:00, 92.8MB/s]
100% 1.32G/1.32G [00:12<00:00, 111MB/s] 


In [ ]:
!unzip -qq gender-recognition-200k-images-celeba.zip 

In [ ]:

import wandb
import numpy as np
import cv2
import tensorflow as tf
from retinaface import RetinaFace
from wandb.keras import WandbCallback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Dense, MaxPool2D, Conv2D, Flatten, BatchNormalization, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import SGD, Adam
from sklearn.metrics import classification_report, confusion_matrix
   

In [ ]:
!pwd

/content


In [ ]:
wandb.init(project="gender-recognition")

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


#Hyper Parameters

In [ ]:
batch_size = 256
epochs = 40
lr = 0.001
width = height = 64

#Dataset


In [ ]:
# data_generator = ImageDataGenerator(
#     rescale=1./255,
#     horizontal_flip=True)

# train_data = data_generator.flow_from_directory("Dataset/Train",
#                                     batch_size = batch_size,
#                                     shuffle = True,
#                                     class_mode = 'binary',
#                                     target_size = (width,height))

# val_data = data_generator.flow_from_directory("Dataset/Validation",
#                                     batch_size = batch_size,
#                                     shuffle = True,
#                                     class_mode = 'binary',
#                                     target_size = (width,height))
# test_data = data_generator.flow_from_directory("Dataset/Test",
#                                     batch_size = batch_size,
#                                     shuffle = False,
#                                     class_mode = 'binary',
#                                     target_size = (width,height))



data_generator = ImageDataGenerator(
                                  rescale = 1 / 255,
                                  horizontal_flip = True
                                  )

train_data = data_generator.flow_from_directory("Dataset/Train/",
                                                    target_size = (width, height),
                                                    class_mode = 'categorical',
                                                    batch_size = batch_size,
                                                    shuffle = True
                                                    )

val_data = data_generator.flow_from_directory("Dataset/Validation/",
                                                    target_size = (width, height),
                                                    class_mode = 'categorical',
                                                    batch_size = batch_size,
                                                    shuffle = True
                                                    )

test_data = data_generator.flow_from_directory("Dataset/Test/",
                                                    target_size = (width, height),
                                                    class_mode = 'categorical',
                                                    batch_size = batch_size,
                                                    shuffle = False
                                                    )

Found 160000 images belonging to 2 classes.
Found 22598 images belonging to 2 classes.
Found 20001 images belonging to 2 classes.


In [ ]:
# model = tf.keras.models.Sequential([
#       #1st conv
#       layers.Conv2D(96,(11,11),activation='relu', input_shape=(width,height,3)),
#       layers.BatchNormalization(),
#       layers.MaxPooling2D(2),

#       #2nd conv
#       tf.keras.layers.Conv2D(256,(11,11),strides=(1,1),activation='relu',padding="same"),
#       tf.keras.layers.BatchNormalization(),


#       #3rd conv
#       tf.keras.layers.Conv2D(384,(3,3),strides=(1,1),activation='relu',padding="same"),
#       tf.keras.layers.BatchNormalization(),


#       #4th conv
#       tf.keras.layers.Conv2D(384,(3,3),strides=(1,1),activation='relu',padding="same"),
#       tf.keras.layers.BatchNormalization(),



#       #5st conv
#       tf.keras.layers.Conv2D(256,(3,3),strides=(1,1),activation='relu',padding="same"),
#       tf.keras.layers.BatchNormalization(),
#       tf.keras.layers.MaxPooling2D(2,strides=(2,2)),


#       layers.Flatten(),


#       #MLP , Classsification

#       #To FC layer 1
#       tf.keras.layers.Dense(4096,activation='relu'),
#       tf.keras.layers.Dropout(0.5),

#       #To FC layer 2
#       tf.keras.layers.Dense(4096,activation='relu'),
#       tf.keras.layers.Dropout(0.5),
#       tf.keras.layers.Dense(1,activation='sigmoid'),

# ]) 





model = tf.keras.models.Sequential([

    Conv2D(96, (11, 11), activation= 'relu', input_shape = (width, height, 3)),
    BatchNormalization(),
    MaxPooling2D(2),


    Conv2D(256, (11, 11), strides=(1, 1), activation= 'relu', padding= 'same'),
    BatchNormalization(),


    Conv2D(384, (3, 3), strides=(1, 1), activation= 'relu', padding= 'same'),
    BatchNormalization(),


    Conv2D(384, (3, 3), strides=(1, 1), activation= 'relu', padding= 'same'),
    BatchNormalization(),


    Conv2D(256, (3, 3), strides=(1, 1), activation= 'relu', padding= 'same'),
    BatchNormalization(),
    MaxPooling2D(2, strides= (2, 2)),


    Flatten(),


    Dense(4096, activation= 'relu'),
    Dropout(0.5),


    Dense(4096, activation= 'relu'),
    Dropout(0.5),


    Dense(2, activation='sigmoid')                          
])

In [ ]:
base_model = tf.keras.applications.ResNet50V2(
    input_shape=(width, height, 3),
    include_top = False,
    weights = 'imagenet',
    pooling = 'max'
    )

In [ ]:
for layer in base_model.layers[:-8]:
  layer.trainable = False

In [ ]:
model = tf.keras.models.Sequential([
    base_model,
    Dense(2, activation='softmax')                          
])

In [ ]:
model.compile(optimizer = tf.keras.optimizers.SGD(),
              loss = tf.keras.losses.categorical_crossentropy,
              metrics='accuracy')

In [ ]:
model.fit(train_data,
          steps_per_epoch = train_data.samples / batch_size,
          validation_data = val_data,
          validation_steps = val_data.samples / batch_size,
          epochs = 5,
          callbacks = [WandbCallback()]
)

Epoch 1/5
625/625 [==============================] - 246s 334ms/step - loss: 0.3472 - accuracy: 0.8515 - val_loss: 0.2869 - val_accuracy: 0.8779
Epoch 2/5
625/625 [==============================] - 203s 325ms/step - loss: 0.2675 - accuracy: 0.8861 - val_loss: 0.2786 - val_accuracy: 0.8814
Epoch 3/5
625/625 [==============================] - 201s 322ms/step - loss: 0.2431 - accuracy: 0.8989 - val_loss: 0.2755 - val_accuracy: 0.8834
Epoch 4/5
625/625 [==============================] - 204s 327ms/step - loss: 0.2267 - accuracy: 0.9054 - val_loss: 0.2697 - val_accuracy: 0.8884
Epoch 5/5
625/625 [==============================] - 203s 325ms/step - loss: 0.2100 - accuracy: 0.9141 - val_loss: 0.2670 - val_accuracy: 0.8884


In [ ]:
model.save('/content/drive/MyDrive/gender_model.h5')

In [ ]:

Y_pred = model.predict(test_data)
y_pred = np.argmax(Y_pred, axis= 1)

print(confusion_matrix(test_data.classes, y_pred))
targets = set(test_data.class_indices.keys())
print('classification_report:')
print(classification_report(test_data.classes, y_pred, target_names= targets))

[[10648   894]
 [ 1226  7233]]
classification_report:
              precision    recall  f1-score   support

      Female       0.90      0.92      0.91     11542
        Male       0.89      0.86      0.87      8459

    accuracy                           0.89     20001
   macro avg       0.89      0.89      0.89     20001
weighted avg       0.89      0.89      0.89     20001



In [ ]:

def process_and_predict(image_path):
    faces = RetinaFace.extract_faces(img_path= image_path, align= True)
    image = faces[0]
    image = cv2.resize(image, (width, height))
    plt.imshow(image)
    image = image / 255.0
    image = np.expand_dims(image, axis= 0)
    res = np.argmax(model.predict(image))
    if res == 0:
        print('Female\n')
    else:
        print('Male\n')